In [2]:
!conda install pyqt

Active code page: 65001
^C


In [1]:
import sys
from PyQt5.QtWidgets import *
#from PyQt5.QtWidgets import QApplication, QWidget, QLabel
from PyQt5.QtGui import QIcon
from PyQt5.QtCore import pyqtSlot
from PyQt5 import uic

def func(ui):
   info = ui.comment.toPlainText()
   alert = QMessageBox()
   alert.setText(info)
   alert.exec()

app = QApplication([])
ui = uic.loadUi("test.ui")
ui.button.clicked.connect(lambda:func(ui))
# def window():
#    flag = 0
#    app = QApplication([])
#    ui = uic.loadUi("test.ui")
#    ui.show()
#    ui.button.clicked.connect(lambda:func(ui,flag))
#    ui.button.show()
#    app.exec()



ui.button.show()
app.exec()




In [1]:
from PyQt5.QtWidgets import *

app = QApplication([])
button = QPushButton('Click')

def on_button_clicked():
    alert = QMessageBox()
    alert.setText('You clicked the button!')
    alert.exec()

button.clicked.connect(on_button_clicked)
button.show()
app.exec()

0

In [1]:
from PySide2.QtWidgets import QApplication, QMessageBox
from PySide2.QtUiTools import QUiLoader

class Stats:

    def __init__(self):
        # 从文件中加载UI定义

        # 从 UI 定义中动态 创建一个相应的窗口对象
        # 注意：里面的控件对象也成为窗口对象的属性了
        # 比如 self.ui.button , self.ui.textEdit
        self.ui = QUiLoader().load('test.ui')

        self.ui.button.clicked.connect(self.handleCalc)

    def handleCalc(self):
        info = self.ui.comment.toPlainText()

        salary_above_20k = ''
        salary_below_20k = ''
        for line in info.splitlines():
            if not line.strip():
                continue
            parts = line.split(' ')

            parts = [p for p in parts if p]
            name,salary,age = parts
            if int(salary) >= 20000:
                salary_above_20k += name + '\n'
            else:
                salary_below_20k += name + '\n'

        QMessageBox.about(self.ui,
                    '统计结果',
                    f'''薪资20000 以上的有：\n{salary_above_20k}
                    \n薪资20000 以下的有：\n{salary_below_20k}'''
                    )

app = QApplication([])
stats = Stats()
stats.ui.show()
app.exec_()

: 

: 

In [1]:
import os
import math
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from opencc import OpenCC

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
#from keras.layers.normalization import layer_normalization
from tensorflow.keras.layers import LayerNormalization
from keras.preprocessing.sequence import pad_sequences
from transformers import BertTokenizer, BertConfig, BertModel

In [2]:
bert_path = './chinese_roberta_wwm_ext_pytorch'
tokenizer = BertTokenizer(f'{bert_path}/vocab.txt')
config = BertConfig.from_pretrained(f'{bert_path}/bert_config.json')
embedding = BertModel.from_pretrained(f'{bert_path}/pytorch_model.bin', config=config)

class bert_model(nn.Module):
    def __init__(self, embedding):
        super(bert_model, self).__init__()
        self.embedding = embedding
        self.fc1 = nn.Linear(768, 2)
        self.fc2 = nn.Linear(768, 5)
    def forward(self, x):
        embedded = self.embedding(x)['pooler_output']
        out1 = self.fc1(embedded)
        out2 = self.fc2(embedded)
        return out1, out2
model = bert_model(embedding).cpu()

model.load_state_dict(torch.load(f'{bert_path}/best_model1.pth'))

Some weights of the model checkpoint at ./chinese_roberta_wwm_ext_pytorch/pytorch_model.bin were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

In [3]:
from PyQt5.QtWidgets import *

class Stats():
    def __init__(self):
        self.window = QMainWindow()
        self.window.resize(500, 400)
        self.window.move(300, 300)
        self.window.setWindowTitle('Goggle留言大解密')

        self.textEdit = QPlainTextEdit(self.window)
        self.textEdit.setPlaceholderText("請輸入一則留言內容")
        self.textEdit.move(10, 25)
        self.textEdit.resize(300, 350)

        self.button = QPushButton('分析', self.window)
        self.button.move(380, 80)

        self.button.clicked.connect(self.func)

    def func(self):
        m = nn.Softmax(dim=1)
        cc = OpenCC('s2t')
        type_names = ['attraction,   ', 'bookstore,  ', 'medical,      ', 'restaurant,  ', 'shopping,   ']
        
        
        info = self.textEdit.toPlainText()
        info = cc.convert(info)
        info = tokenizer.encode(info)
        info = pad_sequences([info], maxlen=100, padding='post',dtype='int')
        info = torch.tensor(info)
        
        pred1,pred2 = model(info)
        pred1 = m(pred1).detach().numpy().squeeze()
        pred2 = list(m(pred2).detach().numpy().squeeze())
        tmp_dict = dict(zip(type_names, pred2))
        tmp_dict= dict(sorted(tmp_dict.items(), key=lambda item: item[1],reverse=True))
        place_res = ""
        for idx,it in enumerate(tmp_dict.items()):
            place_res += "top{}: {} 機率:{:.3f}\n".format(idx+1,it[0],it[1])
        
        str0="語意分析結果:\n"
        str1="              正面   負面\n"
        str2="機率: {:8.3f} {:.3f}\n\n".format(pred1[1],pred1[0])
        str3="地點分類結果:\n"
        
        
        st = str0 + str1 + str2 + str3 + place_res
        print(place_res)
        QMessageBox.about(self.window,
                          "結果:",st)
    

app = QApplication([])
stats = Stats()
stats.window.show()
app.exec_()

top1: attraction,    機率:0.460
top2: shopping,    機率:0.301
top3: restaurant,   機率:0.155
top4: medical,       機率:0.048
top5: bookstore,   機率:0.036

top1: attraction,    機率:0.495
top2: shopping,    機率:0.336
top3: restaurant,   機率:0.111
top4: medical,       機率:0.031
top5: bookstore,   機率:0.028

top1: attraction,    機率:0.436
top2: shopping,    機率:0.335
top3: restaurant,   機率:0.147
top4: medical,       機率:0.053
top5: bookstore,   機率:0.030

top1: attraction,    機率:0.551
top2: shopping,    機率:0.285
top3: restaurant,   機率:0.112
top4: medical,       機率:0.029
top5: bookstore,   機率:0.024

top1: restaurant,   機率:0.749
top2: shopping,    機率:0.125
top3: attraction,    機率:0.119
top4: bookstore,   機率:0.006
top5: medical,       機率:0.002

top1: restaurant,   機率:0.803
top2: shopping,    機率:0.116
top3: attraction,    機率:0.074
top4: bookstore,   機率:0.006
top5: medical,       機率:0.001

top1: restaurant,   機率:0.834
top2: shopping,    機率:0.103
top3: attraction,    機率:0.055
top4: bookstore,   機率:0.006
top5: med

0

In [1]:
import sys
from PyQt5.QtWidgets import (QApplication, QWidget)

class MyWidget(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        self.setWindowTitle('hello world')
        self.setGeometry(50, 50, 200, 150)

if __name__ == '__main__':
    app = QApplication(sys.argv)
    w = MyWidget()
    w.show()
    sys.exit(app.exec_())

SystemExit: 0

c:\Users\USER\anaconda3\envs\back\lib\site-packages\IPython\core\interactiveshell.py:3369: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
